<a href="https://colab.research.google.com/github/randomfunction/DLcpp/blob/main/TTNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Vanilla Two-Tower Neural Network Arch

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [4]:
class Tower(nn.Module):
  def __init__(self, inputdim, embeddingdim):
    super(Tower, self).__init__()
    self.fc1= nn.Linear(inputdim, 256)
    self.fc2= nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 64 )
    self.fc4= nn.Linear(64, embeddingdim)

  def forward(self,x):
    x= F.relu(self.fc1(x))
    x= F.relu(self.fc2(x))
    x= F.relu(self.fc3(x))
    x= self.fc4(x)
    return x

In [5]:
class TwotowerModel(nn.Module):
  def __init__(self, DIM1, DIM2, embeddingdim):
    super(TwotowerModel, self).__init__()
    self.tower1= Tower(DIM1, embeddingdim)
    self.tower2= Tower(DIM2, embeddingdim)

  def forward(self, INPUT1, INPUT2):
    embedding1= self.tower1(INPUT1)
    embedding2= self.tower2(INPUT2)

    #normalize
    embedding1= F.normalize(embedding1)
    embedding2= F.normalize(embedding2)

    #similarity
    similarity= F.cosine_similarity(embedding1, embedding2)

    return embedding1, embedding2, similarity

In [6]:
DIM1=50
DIM2=50
embeddingdim=10
model= TwotowerModel(DIM1, DIM2, embeddingdim)
INPUT1= torch.randn(10, DIM1)
INPUT2= torch.randn(10, DIM2)
embedding1, embedding2, similarity= model(INPUT1, INPUT2)
print(embedding1)
print(embedding2)
print(similarity)


tensor([[-0.1272, -0.3198, -0.5427, -0.3400, -0.1266, -0.2701,  0.2154, -0.2980,
         -0.1250,  0.4813],
        [-0.3282, -0.3456, -0.6336, -0.3675, -0.0232, -0.2068,  0.2456, -0.1806,
         -0.0957,  0.3017],
        [-0.1893, -0.3855, -0.5091, -0.2711,  0.0639, -0.1892,  0.2393, -0.2996,
          0.1068,  0.5335],
        [-0.2340, -0.4451, -0.5838, -0.3110, -0.0409, -0.3466,  0.0954, -0.2687,
          0.1082,  0.3078],
        [-0.3668, -0.4783, -0.5141, -0.2265,  0.1370, -0.2935,  0.0894, -0.3363,
          0.1181,  0.2846],
        [-0.2762, -0.4620, -0.4832, -0.1888, -0.0046, -0.4247,  0.1071, -0.3500,
          0.0187,  0.3555],
        [-0.1930, -0.3888, -0.4973, -0.1231,  0.0364, -0.3656,  0.1993, -0.3863,
         -0.0237,  0.4739],
        [-0.1724, -0.4749, -0.5442, -0.2515,  0.0204, -0.2607,  0.1673, -0.2879,
         -0.0964,  0.4435],
        [-0.1962, -0.3255, -0.4790, -0.1768,  0.0069, -0.3213,  0.3095, -0.4265,
         -0.0269,  0.4616],
        [-0.2319, -

# CNN Two-Tower NN arch.

In [7]:
""" Everything quite similar we just replace fully connected layers
     of Tower class , with some 1D convlution layers
"""

class CNNTower(nn.Module):
  def __init__(self, inputdim, embeddingdim):
    super(CNNTower, self).__init__()
    self.conv1= nn.Conv1d(inputdim, 128, kernel_size=3, padding=1)
    self.conv2= nn.Conv1d(128, 64, kernel_size=3, padding=1)
    self.conv3= nn.Conv1d(64, 32, kernel_size=3, padding=1)
    self.fc4= nn.Linear(32, embeddingdim)

  def forward(self,x):
    x= F.relu(self.conv1(x))
    x= F.relu(self.conv2(x))
    x= F.relu(self.conv3(x))
    x= x.view(x.size(0),-1) #flatten
    x= self.fc4(x)
    return x


# CNN + RNN  Two-Tower NN arch.

In [8]:
"""
We can combine CNN + RNN for better accuracy as,
CNN good at capturing local, high level features while RNN captures temporal dependencies, long term trends.
"""

class CNN_RNN_Tower(nn.Module):
  def __init__(self, inputdim, embeddingdim):
    super(CNN_RNN_Tower, self).__init__()
    self.conv1= nn.Conv1d(inputdim, 128, kernel_size=3, padding=1)
    self.conv2= nn.Conv1d(128, 64, kernel_size=3, padding=1)
    self.conv3= nn.Conv1d(64, 32, kernel_size=3, padding=1)
    self.rnn= nn.LSTM(32, 64, num_layers=2, batch_first=True)
    self.fc4= nn.Linear(64, embeddingdim)

  def forward(self,x):
    x= F.relu(self.conv1(x))
    x= F.relu(self.conv2(x))
    x= F.relu(self.conv3(x))
    x= x.view(x.size(0),-1) #flatten
    x= F.relu(self.rnn(x))
    x= self.fc4(x)
    return x
